## <center> Scraping Movies </center>

Prerequisites:
* Make sure to place chromedriver.exe in the same directory as your code, in "\chromedriver-win64" subfolder.
* Make sure you have mysql installed. 
* Make sure you have a database named "scraping". Otherwise, execute this command.
<br>CREATE DATABASE scraping;
* Make sure you have table named "movie" in scraping database. Otherwise, execute this command.
<br>
<span style="color:green;">CREATE TABLE movie (movieid VARCHAR(255) PRIMARY KEY, movie_title VARCHAR(255), movie_year INT, movie_release_date DATE, movie_genre VARCHAR(255), movie_rating INT, movie_href VARCHAR(255), movie_desc TEXT, movie_cast TEXT, movie_tag TEXT, budget DECIMAL(15, 2), revenue DECIMAL(15, 2), page_count INT, download_flag INT);</span>
* Make sure you have table named "rating" in scraping database. Otherwise, execute this command.
<br>
<span style="color:green;">CREATE TABLE rating (reviewid VARCHAR(255) PRIMARY KEY, userid VARCHAR(255), movieid VARCHAR(255), movie_title VARCHAR(255), rating INT, review_date DATE, review_text TEXT, user_href VARCHAR(255), review_href VARCHAR(255), download_flag INT);</span>
* Make sure you have table named "user" in scraping database. Otherwise, execute this command.
<br>
<span style="color:green;">CREATE TABLE user (userid VARCHAR(50) PRIMARY KEY, user_name VARCHAR(100), user_href VARCHAR(255), user_join_date DATE);</span>

In [1]:
import pandas as pd
import re
import os
from tqdm.notebook import tqdm
from datetime import datetime
import time

import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

from selenium import webdriver 
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By

import pymysql

In [2]:
def getPage(driver, url):
    driver.get(url)
    time.sleep(5) # Wait for the page to load, also respecting TMDB rate limit: 40 request per 10 seconds
    bsObj = BeautifulSoup(driver.page_source, 'lxml') # Get the page source and parse it with Beautiful Soup
    return bsObj

def download_Movies (driver, conn, cur, main_url, max_count):
    
    # Movie criteria to download: English movies, sort by revenue
    # Sample string https://www.themoviedb.org/movie?with_original_language=en&sort_by=release_date.desc&release_date.gte=2023-01-01&release_date.lte=2024-10-27&page=1
    # Query string for genre -> &with_genres=27
    # Query string for sort -> &sort_by=popularity.desc, &sort_by=release_date.desc
    # Query string for release date range -> &release_date.gte=2023-01-01&release_date.lte=2024-10-27

    sub_url = main_url + "/movie?with_original_language=en&sort_by=revenue.desc&page="

    page_count = 0
    download_flag = 0
    count = 0
    progress_bar = tqdm(desc='Movie download progress', total=max_count)  # progress bar

    check_dupe_df = select_movieid (cur)

    while count < max_count:
        page_count +=1
        url = sub_url + str(page_count)
        # print(url)
        
        # Open the page
        bsObj = getPage(driver, url)

        div_tags = bsObj.find_all("div", {"class":"card style_1"})

        for div_tag in div_tags:
            try:
                a_tag = div_tag.find('a', {"class": "image"})
                # print(a_tag)

                # Populate movie id, title, link
                movie_href = a_tag.get('href').strip()
                movieid = re.sub('/movie/','', movie_href).strip()

                movie_title = ""
                movie_year = ""
                movie_release_date = ""
                movie_genre = ""
                movie_rating = 0 
                movie_desc = ""
                movie_cast = ""
                movie_tag = ""
                budget = 0.0
                revenue = 0.0

                movie_title = a_tag.get('title').strip()
                movie_title = re.sub(r"['\"]", r"\\'", movie_title)
                # print("Movie Title:", movie_title)

                movie_release_date = div_tag.find('p').get_text()
                if movie_release_date:
                    movie_release_date = datetime.strptime(movie_release_date, "%b %d, %Y").date()
                    movie_release_date = movie_release_date.strftime("%Y-%m-%d")
                # print(movie_release_date)

                # Populate movie's year of release
                bsObj_movie_page = getPage(driver, main_url + movie_href)
                span_tag = bsObj_movie_page.find('span', {"class":"tag release_date"})
                movie_year = re.sub(r'[()]','',span_tag.get_text()).strip()
                # print(movie_year)

                # Populate movie genres
                span_tag = bsObj_movie_page.find('span', {"class":"genres"})
                a_tags = span_tag.find_all("a")
                for a_tag in a_tags:
                    movie_genre = movie_genre + a_tag.get_text() + "|"
                    movie_genre = movie_genre.strip()
                # print(movie_genre)

                # Populate movie rating
                div_tag = bsObj_movie_page.find('div', {"class":"user_score_chart"})
                if div_tag:
                    movie_rating = div_tag.get('data-percent').strip()
                # print(movie_rating)
                
                # Populate movie description
                div_tag = bsObj_movie_page.find('div', {"class":"overview"})
                if div_tag:
                    movie_desc = div_tag.get_text().strip()
                    movie_desc = re.sub(r"['\"]", r"\\'", movie_desc)
                # print(movie_desc)

                # Populate casts
                div_tag = bsObj_movie_page.find('ol', {"class":"people scroller"})
                if div_tag:
                    a_tags = div_tag.find_all('a', href=lambda href: href and '/person/' in href)
                    a_tags = [tag for tag in a_tags if tag.get_text(strip=True)]
                    movie_cast = ""
                    if a_tags:
                        for tag in a_tags:
                            movie_cast = movie_cast + tag.get_text().strip()+ "|"
                # print(movie_cast)

                # Populate keywords
                # div_tag = bsObj_movie_page.find('section', {"class":"keywords right_column"})
                a_tags = bsObj_movie_page.find_all('a', href=lambda href: href and '/keyword/' in href)
                a_tags = [tag for tag in a_tags if tag.get_text(strip=True)]
                movie_tag = ""
                if a_tags:
                    for tag in a_tags:
                        movie_tag = movie_tag + tag.get_text().strip()+ "|"
                # print(movie_tag)
                
                # Populate revenue & budget
                div_tag = bsObj_movie_page.find('section', {"class":"facts left_column"})
                p_tags = div_tag.find_all('p')
                budget_pattern = r"Budget\s*\$(\d{1,3}(?:,\d{3})*(?:\.\d{2})?|\d+\.\d{2})"
                revenue_pattern = r"Revenue\s*\$(\d{1,3}(?:,\d{3})*(?:\.\d{2})?|\d+\.\d{2})"
                if p_tags:
                    tag_text = " ".join(tag.get_text() for tag in p_tags)
                    budget_match = re.findall(budget_pattern, tag_text)
                    budget = float(budget_match[0].replace(",", "")) if budget_match else 0.0
                    revenue_match = re.findall(revenue_pattern, tag_text)
                    revenue = float(revenue_match[0].replace(",", "")) if revenue_match else 0.0
                # print(f"Budget: {budget}, Revenue: {revenue}")

                # print(movieid, movie_title, movie_year, movie_release_date, movie_genre, movie_rating, movie_href, movie_desc, movie_cast, movie_tag, budget, revenue)

                # Insert new record to database
                insert_update_movie_to_db(cur, movieid, movie_title, movie_year, movie_release_date, movie_genre, movie_rating, movie_href, movie_desc, movie_cast, movie_tag, budget, revenue, page_count, download_flag)
                conn.commit()

                # if movieid not in check_dupe_df['movieid'].values:
                count += 1
                time.sleep(2)
                progress_bar.update(1)

                if count >= max_count:
                    break
           
            except Exception as e:
                print(f"Error processing {movie_href}: {e}")
                print(movieid, movie_title, movie_year, movie_release_date, movie_genre, movie_rating, movie_href, movie_desc, movie_cast, movie_tag, budget, revenue)
                continue

    progress_bar.close()
    movie_df = select_all_movie (cur)

    return movie_df

def insert_update_movie_to_db (cursor, movieid, movie_title, movie_year, movie_release_date, movie_genre, movie_rating, movie_href, movie_desc, movie_cast, movie_tag, budget, revenue, page_count, download_flag):
    # sql = '''
    # INSERT INTO movie (movieid, movie_title, movie_year, movie_release_date, movie_genre, movie_rating, movie_href, movie_desc, movie_cast, movie_tag, budget, revenue, page_count, download_flag)
    # SELECT %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
    # FROM DUAL
    # WHERE NOT EXISTS (SELECT 1 FROM movie WHERE movieid = %s);
    # '''
    sql = '''
    INSERT INTO movie (movieid, movie_title, movie_year, movie_release_date, movie_genre, movie_rating, movie_href, movie_desc, movie_cast, movie_tag, budget, revenue, page_count, download_flag)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        movie_title = values(movie_title),
        movie_year = values(movie_year),
        movie_release_date = values(movie_release_date),
        movie_genre = values(movie_genre),
        movie_rating = values(movie_rating),
        movie_href = values(movie_href),
        movie_desc = values(movie_desc),
        movie_cast = values(movie_cast),
        movie_tag = values(movie_tag),
        budget = values(budget),
        revenue = values(revenue);
    '''
    cursor.execute ("USE scraping;")
    cursor.execute(sql, (movieid, movie_title, movie_year, movie_release_date, movie_genre, movie_rating, movie_href, movie_desc, movie_cast, movie_tag, budget, revenue, page_count, download_flag))

def select_all_movie (cursor):
    cursor.execute ("USE scraping;")
    cursor.execute("SELECT * from movie order by page_count, movieid;")
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    df_mysql = pd.DataFrame(rows, columns=columns)
    return df_mysql

def select_movieid (cursor):
    cursor.execute ("USE scraping;")
    cursor.execute("SELECT movieid from movie;")
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    df_mysql = pd.DataFrame(rows, columns=columns)
    return df_mysql

In [3]:
main_url = "https://www.themoviedb.org"
user_agent = {"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"}

# Set up Chrome options
chrome_options = Options()
# chrome_options.add_argument("--headless=new")
# chrome_options.add_argument("--window-position=-2400,-2400") # hide window
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--disable-web-security")
chrome_options.add_argument("--disable-site-isolation-trials")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-blink-features=BlockCredentialedSubresources")
chrome_options.add_argument(f"user-agent={user_agent}")

# Set up the WebDriver
chrome_driver = os.getcwd() + "\\chromedriver-win64" + "\\chromedriver.exe"
service = Service(chrome_driver)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Connect to database
try:
    conn = pymysql.connect(host='127.0.0.1', user='root', passwd='root', db='scraping')
    cur = conn.cursor()
    print("Connection successful!")
except pymysql.MySQLError as e:
    print(f"Error connecting to the database: {e}")

Connection successful!


In [4]:
movie_df = download_Movies (driver, conn, cur, main_url, 5) # change the number to the max number of movies to download
print(movie_df.shape)
display(movie_df.columns)
display(movie_df.head(1))

# Export movie to csv
movie_df.to_csv("movie.csv", index=False)

Movie download progress:   0%|          | 0/5 [00:00<?, ?it/s]

(5000, 14)


Index(['movieid', 'movie_title', 'movie_year', 'movie_release_date',
       'movie_genre', 'movie_rating', 'movie_href', 'movie_desc', 'movie_cast',
       'movie_tag', 'budget', 'revenue', 'page_count', 'download_flag'],
      dtype='object')

,movieid,movie_title,movie_year,movie_release_date,movie_genre,movie_rating,movie_href,movie_desc,movie_cast,movie_tag,budget,revenue,page_count,download_flag
0,1022789-inside-out-2,Inside Out 2,2024,2024-06-13,Animation|Family|Fantasy|Adventure|Comedy|,76,/movie/1022789-inside-out-2,Teenager Riley\'s mind headquarters is undergo...,Amy Poehler|Maya Hawke|Kensington Tallman|Liza...,sadness|disgust|sequel|computer animation|teen...,200000000.00,1682636477.00,1,1


In [5]:
cur.close()
conn.close()
driver.close
driver.quit

<bound method ChromiumDriver.quit of <selenium.webdriver.chrome.webdriver.WebDriver (session="298fd1669a0ed8b56d55ccb3bd64f8db")>>